# Generador de personas aleatorias

In [7]:
import math
import random as rm
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import re
from faker import Faker
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
from typing import Union
from time import sleep

# Obtener complexion fisica segun IMC
def complexion_fisica(altura, peso):
	# peso en kg y altura en m
	imc = peso / (altura ** 2)
	# OJO el imc no distigue sexo, edad, grasa o musculo
	# Digamos que esto define el volumen de la persona
	complexion = ''
	if -np.inf <= imc < 25: complexion = 'Chico'
	elif 25 <= imc < 35: complexion = 'Medio'
	elif 35 <= imc < np.inf: complexion = 'Grande'
	return complexion

# Calcular CURP
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		curp += clave_estados[estado_nac]
	else: curp += 'NE'
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

# Obtener tu signo segun tu cumpleaños
def sacar_signo(fecha_nac: dt.datetime):
	dia = int(fecha_nac.strftime('%d'))
	mes = int(fecha_nac.strftime('%m'))
	astro_sign = ''
	if mes == 12: astro_sign = 'Sagitario' if dia < 22 else 'Capricornio'
	elif mes == 1: astro_sign = 'Capricornio' if dia < 20 else 'Aquario'
	elif mes == 2: astro_sign = 'Aquario' if dia < 19 else 'Piscis'
	elif mes == 3: astro_sign = 'Piscis' if dia < 21 else 'Aries'
	elif mes == 4: astro_sign = 'Aries' if dia < 20 else 'Tauro'
	elif mes == 5: astro_sign = 'Tauro' if dia < 21 else 'Geminis'
	elif mes == 6: astro_sign = 'Geminis' if dia < 21 else 'Cancer'
	elif mes == 7: astro_sign = 'Cancer' if dia < 23 else 'Leo'
	elif mes == 8: astro_sign = 'Leo' if dia < 23 else 'Virgo'
	elif mes == 9: astro_sign = 'Virgo' if dia < 23 else 'Libra'
	elif mes == 10: astro_sign = 'Libra' if dia < 23 else 'Escorpio'
	elif mes == 11: astro_sign = 'Escorpio' if dia < 22 else 'Sagitario'
	return astro_sign

# Instancia para usar servicio de geoubicaciones
geolocator = Nominatim(user_agent = 'geoapiExercises')
geocode = lambda query: geolocator.geocode(query, featuretype = 'city', language = 'es')
reverse = lambda coords: geolocator.reverse(coords, language = 'es')

# Obtener coordenadas del centro de una ciudad
def obtener_coordenadas(ciudad, estado, pais = 'Mexico') -> dict:
	print(f'\033[36mBuscando coordenadas de {ciudad}, {estado}, {pais}\033[0m')
	try:
		# Hacer peticion a un api para obtener coordenadas del centro y extremas
		location = geocode(f'{ciudad} {estado} {pais}')
		# En caso de que no se haya encontrado nada
		if not location: raise BaseException(f'No se encontraron coordenadas')
		location = location.raw
		# Obtener coordenadas del centro
		centro = (float(location['lat']), float(location['lon']))
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in location['boundingbox']]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return {}
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

# Generar n puntos random alrededor de un area
def generar_puntos_random(coords: dict, n = 1, reducir = True) -> list:
	# Ver si estan los 3 puntos necesarios
	coords_necesarias = ('centro', 'suroeste', 'noreste')
	if not all(k in coords for k in coords_necesarias):
		print('\033[33mgenerar_puntos_random()\033[0m: No se encontro alguna de las coordenadas necesarias:', coords_necesarias)
		return []
	# Obtener coordenadas del centro
	cenx, ceny = coords['centro']
	# Obtener extremos originales
	minx, miny = coords['suroeste']
	maxx, maxy = coords['noreste']
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

# Buscar una direccion a partir de una lista de corrdenadas
def buscar_direccion(coords: dict) -> dict:
	# Generar puntos aleatorios alrededor del area
	puntos_random = generar_puntos_random(coords, 5)
	if not puntos_random: return {}
	dirrr = {}
	listo = False
	print('\033[36mBuscando direccion\033[0m')
	for punto_random in puntos_random:
		dirr = {}
		try:
			address = reverse(punto_random)
			if not address: continue
			direccion = address.raw['address']
			print(direccion)
			# Ver que campos se encontraron
			# https://nominatim.org/release-docs/latest/api/Output/#addressdetails
			# local
			if any(k in direccion for k in ('amenity', 'building', 'shop', 'office', 'leisure')):
				if 'amenity' in direccion: dirr['local'] = direccion['amenity']
				elif 'building' in direccion: dirr['local'] = direccion['building']
				elif 'shop' in direccion: dirr['local'] = direccion['shop']
				elif 'office' in direccion: dirr['local'] = direccion['office']
				elif 'leisure' in direccion: dirr['local'] = direccion['leisure']
			# calle
			if 'road' in direccion: dirr['calle'] = direccion['road']
			# numero
			if 'house_number' in direccion: dirr['numero'] = direccion['house_number']
			else: dirr['numero'] = 'S/N'
			# colonia
			if any(k in direccion for k in ('neighbourhood', 'residential', 'suburb', 'quarter')):
				if 'neighbourhood' in direccion: dirr['colonia'] = direccion['neighbourhood']
				elif 'residential' in direccion: dirr['colonia'] = direccion['residential']
				elif 'suburb' in direccion: dirr['colonia'] = direccion['suburb']
				elif 'quarter' in direccion: dirr['colonia'] = direccion['quarter']
			# codigo postal
			if 'postcode' in direccion: dirr['cp'] = direccion['postcode']
			# localidad
			if any(k in direccion for k in ('city', 'town', 'village', 'hamlet')):
				if 'city' in direccion: dirr['localidad'] = direccion['city']
				elif 'town' in direccion: dirr['localidad'] = direccion['town']
				elif 'village' in direccion: dirr['localidad'] = direccion['village']
				elif 'hamlet' in direccion: dirr['localidad'] = direccion['hamlet']
			# municipio
			if 'county' in direccion: dirr['municipio'] = direccion['county']
			# estado
			if 'state' in direccion: dirr['estado'] = direccion['state']
			# pais
			if 'country' in direccion: dirr['pais'] = direccion['country']
			# Si se encontro al menos calle y cp
			if all(k in dirr for k in ('calle', 'numero')): listo = True
			# Si ya se encontro una buena direccion, dejar de buscar
			if listo:
				print('\033[32mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
				print('\033[32m', dirr, '\033[m')
				dirrr = dirr
				break
		except BaseException as e:
			print(f'\033[31mbuscar_direccion()\033[0m:', e)
			return {}
	if not listo: print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion precisa')
	return dirrr

# grados a radianes
def deg2rad(degrees): return math.pi * degrees / 180.0

# radianes a grados
def rad2deg(radians): return 180.0 * radians / math.pi

# Calcular radio de la tierra (R) en metros con una latitud dada segun WGS-84
def radio_terrestre_wgs84(lat: float) -> float:
	# http://en.wikipedia.org/wiki/Earth_radius
	# Semiejes de WGS-84
	semieje_mayor_a = 6378137.0  # Semieje mayor a [metros]
	semieje_menor_b = 6356752.31424  # Semieje menor b [metros]
	# Sacar R
	an = (semieje_mayor_a ** 2) * math.cos(lat)
	bn = (semieje_menor_b ** 2) * math.sin(lat)
	ad = semieje_mayor_a * math.cos(lat)
	bd = semieje_menor_b * math.sin(lat)
	return math.sqrt((an ** 2 + bn ** 2) / (ad ** 2 + bd ** 2))

# Calcular cuadro delimitador a partir de una cordenada dada y su amplitud
def bounding_box(coords: tuple, amplitud_en_km = 6) -> list:
	# convertir latitud y longitud a radianes
	lat_rad = deg2rad(coords[0])
	lon_rad = deg2rad(coords[1])
	# ampliar el area de busqueda
	amplitud = 1000 * amplitud_en_km

	# Radio terrestre en la latitud dada
	radius = radio_terrestre_wgs84(lat_rad)
	# Radio paralelo en la latitud
	pradius = radius * math.cos(lat_rad)

	# Calcular limites y convertirlos a grados
	lat_min = rad2deg(lat_rad - amplitud / radius)
	lon_min = rad2deg(lon_rad - amplitud / pradius)
	lat_max = rad2deg(lat_rad + amplitud / radius)
	lon_max = rad2deg(lon_rad + amplitud / pradius)
	return [(lat_min, lon_min), (lat_max, lon_max)]

In [18]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
pge = pd.read_csv('./conjunto_datos/porcentajes_grasa_edad.csv')
ct_complx = pd.read_csv('./catalogos/edad_pg_complexion.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for i in range(n):
		print(f'\033[34mIdx{i}\033[0m')
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], [30, 6, 9, 2, 4, 1, 39, 9])[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()), maximum_age = 90)
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		if edad <= reap['edad'].max(): criter = (reap['edad'] == edad)
		else: criter = (reap['edad'] == reap['edad'].max())
		margen_ap = reap[criter].to_dict(orient = 'records')[0]

		# Dependiendo de la edad se busca el rango de % de grasa
		margen_pge = pge[(pge['li.edad'] <= edad) & (pge['ls.edad'] >= edad)].to_dict(orient = 'records')[0]

		# Se filtra por edad los criterios para saber la complexion fisica
		# mas adelante se checara el sexo y en que intervalo esta su % de grasa
		comp_criter = (ct_complx['li.edad'] <= edad) & (ct_complx['ls.edad'] >= edad)

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['li.altura.h.cm'], margen_ap['ls.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['li.peso.h.kg'], margen_ap['ls.peso.h.kg']), 2)
			grasa = round(rm.uniform(margen_pge['li.pg.h'], margen_pge['ls.pg.h']), 1)
			comp_criter = comp_criter & (ct_complx['li.pg.h'] <= grasa) & (ct_complx['ls.pg.h'] >= grasa)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['li.altura.m.cm'], margen_ap['ls.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['li.peso.m.kg'], margen_ap['ls.peso.m.kg']), 2)
			grasa = round(rm.uniform(margen_pge['li.pg.m'], margen_pge['ls.pg.m']), 1)
			comp_criter = comp_criter & (ct_complx['li.pg.m'] <= grasa) & (ct_complx['ls.pg.m'] >= grasa)

		complexion = ct_complx[comp_criter]['complexion'].values[0]
		mins_ejerc_dia = round(rm.normalvariate(60, 70))
		if mins_ejerc_dia < 0: mins_ejerc_dia = 0

		apellido_p = fake.last_name()
		apellido_m = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_i_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		tel = '({:d}) {:03d} {:04d}'.format(ct_ciudad['lada'], rm.randint(0, 999), rm.randint(0, 9999))
		curp = sacar_curp(nombre, apellido_p, apellido_m, sexo, fecha_nac, estado)
		signo = sacar_signo(fecha_nac)
		religion = rm.choices(['Catolico', 'Cristiano', 'Otros', 'Ateo'], [78, 11, 3, 8])[0]
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None

		"""Correo"""
		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_p}',
			# alonso_perez
			3: f'{nombre}_{apellido_p}',
			# aps70
			4: f'{nombre[0]}{apellido_p[0]}{apellido_m[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_p[0]}{apellido_m[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_p}{apellido_m}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos ahi mismo. El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# TODO Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			# suponiendo que la persona empezo a trabajar de sus 18 a sus 25
			nss2 = (fecha_nac + pd.DateOffset(months = rm.randint(216, 300))).strftime('%y')
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%04d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			coords = {
				'centro': eval(ct_ciudad['centro']),
				'suroeste': eval(ct_ciudad['suroeste']),
				'noreste': eval(ct_ciudad['noreste'])
			}
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			coords = obtener_coordenadas(ciudad, estado)
			if coords:
				# guardar coordenadas en el catalogo
				# para que a la otra no haya que buscar en internet
				ct_ciudades.loc[ct_ciudad_i_row, 'centro'] = str(coords['centro'])
				ct_ciudades.loc[ct_ciudad_i_row, 'suroeste'] = str(coords['suroeste'])
				ct_ciudades.loc[ct_ciudad_i_row, 'noreste'] = str(coords['noreste'])
				ct_ciudades.to_csv(file_ciudades, index = False)

		"""Direccion"""
		direcc = cp = None
		if 'centro' in coords:
			direccion = buscar_direccion(coords)
			if direccion:
				cp = direccion['cp'] if 'cp' in direccion else None
				if all(k in direccion for k in ('calle', 'numero')):
					direcc = f"{direccion['calle']}, {direccion['numero']}"
					if 'colonia' in direccion: direcc += f", {direccion['colonia']}"

		"""Zona horaria"""
		timezone = tz_offset = None
		if 'centro' in coords:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = coords['centro'][0], lng = coords['centro'][1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not 'centro' in coords: coords['centro'] = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_p': apellido_p,
			'apellido_m': apellido_m,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'tipo_sangre': tipo_sangre,
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'complexion': complexion,
			'porcent_grasa': grasa,
			'mins_ejerc_dia': mins_ejerc_dia,
			'color_fav': fake.safe_color_name(),
			'signo': signo,
			'religion': religion,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': coords['centro'][0],
			'lon': coords['centro'][1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})

	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx0
Buscando direccion
{'shop': 'Taller Mecanico "Sanchez"', 'road': 'Calle Nicolás Bravo', 'town': 'Nueva Rosita', 'county': 'San Juan de Sabinas', 'state': 'Coahuila', 'ISO3166-2-lvl4': 'MX-COA', 'postcode': '25800', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): Direccion encontrada en (27.943785548140152, -101.22357474120395)
 {'local': 'Taller Mecanico "Sanchez"', 'calle': 'Calle Nicolás Bravo', 'numero': 'S/N', 'cp': '25800', 'localidad': 'Nueva Rosita', 'municipio': 'San Juan de Sabinas', 'estado': 'Coahuila', 'pais': 'México'} 
Idx1
Buscando direccion
{'village': 'La Victoria', 'county': 'Pinos', 'state': 'Zacatecas', 'ISO3166-2-lvl4': 'MX-ZAC', 'country': 'México', 'country_code': 'mx'}
{'village': 'La Victoria', 'county': 'Pinos', 'state': 'Zacatecas', 'ISO3166-2-lvl4': 'MX-ZAC', 'country': 'México', 'country_code': 'mx'}
{'village': 'La Victoria', 'county': 'Pinos', 'state': 'Zacatecas', 'ISO3166-2-lvl4': 'MX-ZAC', 'country': 'México', 'country_code': 'mx'}


,titulo,nombre,apellido_p,apellido_m,fecha_nac,tipo_sangre,sexo,altura,peso,complexion,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,None,Homero,Laureano,Treviño,1967-05-02,O+,H,173.26,76.82,Obeso,...,Astrónomo,32 91 67 7078 9,"Calle Nicolás Bravo, S/N",25800,Nueva Rosita,Coahuila,27.937833,-101.218749,America/Monterrey,-0500
1,None,Aurelio,Lara,Viera,1961-09-16,B+,H,177.33,78.39,Atleta,...,Bailarín,12 85 61 2498 9,None,None,La Victoria,Zacatecas,22.257501,-101.629959,America/Mexico_City,-0500
2,None,José Manuél,Solorzano,Mondragón,1987-02-22,O+,H,185.09,93.68,Media,...,Trabajador social,29 11 87 8294 1,"Calle 25, S/N",97345,Conkal,Yucatán,21.075061,-89.529121,America/Merida,-0500
3,Sra.,Itzel,Caldera,Figueroa,1990-12-04,A+,M,166.29,72.41,Media,...,Desarrollador de aplicaciones,09 12 90 0027 7,"Carretera Mapimí-Bermejillo, S/N",None,Mapimi,Durango,25.833740,-103.848022,America/Monterrey,-0500
4,Sra.,Carmen,Vélez,Cantú,1946-12-29,A+,M,160.37,63.44,Media,...,Oficial de policia municipal,14 66 46 5395 3,"Tehuixtla, S/N",62662,Tehuixtla,Morelos,18.621896,-99.319025,America/Mexico_City,-0500
5,None,Federico,Fajardo,Carrera,1985-09-30,B+,H,178.78,60.74,Obeso,...,Director financiero,21 06 85 7018 1,"Calle Profesora Rosario M. De Zárate, S/N",21980,República Mexicana,Baja California,32.646078,-114.797914,America/Tijuana,-0700
6,None,Ramón,Uribe,Nevárez,2007-02-09,O+,H,167.00,43.10,Obeso,...,None,None,"Calle José Vasconcelos, S/N",28310,Rincón de López,Colima,19.053324,-103.933609,America/Mexico_City,-0500
7,None,Frida,Villalpando,Girón,2000-11-29,O+,M,174.81,77.48,Media,...,CEO,57 19 00 3453 0,"Autopista Guadalajara - Colima, S/N",28550,El Trapiche,Colima,19.293059,-103.649740,America/Mexico_City,-0500
8,Sr.,Ramón,Cintrón,Blanco,1958-07-18,A+,H,189.45,84.05,Obeso,...,Albañil,46 79 58 7869 7,"Carretera La Puerta, S/N",None,Nayarit Llamada,Baja California,32.312666,-115.246546,America/Tijuana,-0700
9,Mtra.,Caridad,Paredes,Roque,1999-08-23,A+,M,166.94,65.97,Bueno,...,Diseñador de cerámica,68 22 99 8606 2,"Calle Gabriel López, S/N",20576,San José de Gracia,Aguascalientes,22.146807,-102.412781,America/Mexico_City,-0500


In [ ]:
# Buscar coordenadas de cada ciudad y crear un csv con las ciudades que no se le pudieron encontrar una
def buscar_ciudades_perdidas():
	ciudades_perdidas = []
	for i, ciudad in ct_ciudades.iterrows():
		# si esta ciudad no tiene estos tres campos
		no_hay_centro = pd.isna(ciudad['centro'])
		no_hay_suroeste = pd.isna(ciudad['suroeste'])
		no_hay_noreste = pd.isna(ciudad['noreste'])
		if (no_hay_centro and no_hay_suroeste and no_hay_noreste) or (no_hay_suroeste and no_hay_noreste):
			print(f'\033[34mIdx{i}\033[0m')
			# para no hacer peticiones muy seguido
			sleep(5)
			# buscar en internet el punto central y extremos segun la ciudad
			coordenadas = obtener_coordenadas(ciudad['ciudad'], ciudad['estado'])
			if coordenadas:
				centro = coordenadas['centro']
				# guardar coordenadas en el catalogo
				# para que a la otra no haya que buscar en internet
				ct_ciudades.loc[i, 'centro'] = str(centro)
				ct_ciudades.loc[i, 'suroeste'] = str(coordenadas['suroeste'])
				ct_ciudades.loc[i, 'noreste'] = str(coordenadas['noreste'])
				ct_ciudades.to_csv(file_ciudades, index = False)
			else:
				ciudades_perdidas.append({
					'idx': i,
					'estado': ciudad['estado'],
					'cuidad': ciudad['ciudad']
				})
	df = pd.DataFrame(ciudades_perdidas)
	df.to_csv('./catalogos/ciudades_perdidas.csv', index = False)
	print('\033[34mListo\033[0m')